In [ ]:
import pandas as p
url = 'https://etherscan.io/txsInternal?a=0xea3f1b409f5e9211eed087c9f5f621d1249b3e7b'
from qore import *
xp = XPath()
xcols = {
    #'to'      : '/html/body/div[1]/div[4]/div[2]/div/div/div/table/tbody/tr/td[6]/span/a/text()',
    'value'   : '/html/body/div[1]/div[4]/div[2]/div/div/div/table/tbody/tr/td[7]/text()',
}
res = xp.xpath2df(url, xcols)
p.DataFrame(res)

In [ ]:
import sys
def defp(pt):
    try:    sys.path.index(pt)
    except: sys.path.append(pt)
#defp('/ml.dev/bin')

from qoreliquid import *

from bitmex import desc
from bitmex import apiRequest
from bitmex import CoinMarketCap
from bitmex import CoinMarketCap
from bitmex import PortfolioModeler

%reload_ext autoreload
%autoreload 2
%pylab inline
import numpy as n

from pylab import rcParams
rcParams['figure.figsize'] = 20, 5
import seaborn as sns
sns.set()

pm = PortfolioModeler()
cmc = CoinMarketCap()

In [ ]:
#none = desc(df)
#pp = sns.pairplot(df)
#plt.show()
#none

In [ ]:
cmc = CoinMarketCap()
dft = cmc.getAllTokens(tokens=False)
#dft
#cmc.tokens

In [ ]:
dft = cmc.getAllTokens(tokens=True)
#dft
#cmc.tokens

In [ ]:
df = pm.generatePortfolioT1Supply(dft, balance=(.12*7800), risk=100)
df#['id']

In [ ]:
#df = df.head(300)
#df[fi] = map(lambda x: x.decode('utf-8').strip(), df[fi])
#pp = sns.pairplot(df)#, hue=fi)
#plt.show()

In [ ]:
def dataFrameAddMultiIndex(dfm, title):
    dfi = dfm.transpose()
    #with p.option_context('display.max_rows', 400, 'display.max_columns', 4000, 'display.width', 1000000):
    #    print dfi
    #    print
    zi   = zip([title]*len(dfi.index), dfi.index)
    indx = p.MultiIndex.from_tuples(zi)
    #print indx
    dfi = p.DataFrame(dfi.to_dict(orient='list'), index=indx)
    dfi = dfi.transpose()#.tail(6)
    #with p.option_context('display.max_rows', 400, 'display.max_columns', 4000, 'display.width', 1000000):
    #    print dfi
    return dfi

def asd(token):
    dfm = cmc.glo(token)
    dfi = dataFrameAddMultiIndex(dfm, token)
    #with p.option_context('display.max_rows', 400, 'display.max_columns', 4000, 'display.width', 1000000):
    #    print dfi.tail(10)
    return dfi

def axs(df, li):
    dfp = p.DataFrame()
    dfp = dfp.transpose()
    dfi = asd('bitcoin')
    dfp = dfp.reindex(index=dfi.transpose().index, level=0).transpose()
    dfp = dfp.combine_first(dfi)
    #dfp = dfp.combine_first(asd('propy'))

    dfp = p.concat([dfp, asd('ethereum')], axis=1)

    for i in li:
        try:    dfp = p.concat([dfp, asd(i)], axis=1)
        except Exception as e: print e

    #dfp = dfi.combine_first(dfp)
    #asd('propy').merge(dfp)
    #p.merge(dfp, asd('propy'), left_index=True, right_index=True)
    #p.concat([dfp, asd('propy')], axis=1, join='outer')
    #p.concat([dfp.transpose(), asd('propy').transpose()], axis=0).sort_index().transpose()
    #dfp = p.concat([dfp, asd('propy')], axis=1).sort_index()
    #dfp = dfp.combine_first(asd('ethereum'))

    #with p.option_context('display.max_rows', 400, 'display.max_columns', 4000, 'display.width', 1000000):
    #    print dfp.loc[[1510942454000,1510942451000], :]
    #    print dfi.tail(10)
    #    print dfe.tail(10)
    dfp = dfp.ffill().bfill()
    dfp = normalizeme(dfp)
    dfp = sigmoidme(dfp)
    #dfp.tail(1000).plot()
    return dfp

def genDFP(cmc, port=None):
    at = cmc.getAllTokens(tokens=False)
    if port:
        port = port.split(' ')
    else:
        port = list(at.loc[:,'id'])
        #print at.loc[port, 'id']
    if port != None:
        dfportfolio = pm.generatePortfolioT1Supply(at)
        li = list(dfportfolio.loc[port,['id']]['id'])
    else:
        dfportfolio = pm.generatePortfolioT1Supply(at)
        li = list(dfportfolio.loc[:,['id']]['id'])
    #li = list(df['id'].head(30))
    #li = 'ethereum propy bitcoin-cash ripple dash iota monero neo'.split(' ')
    dfp = axs(df, li)
    return dfp

#port = 'PPP XRL BBT SALT PPT TAAS SIFT PAY OMG KIN AIR PLR LINK RHOC CDT DNT RDN POWR CAT ENG PRO KNC SND AVT HMQ MGO ZRX VERI CVC EOS'
#port = 'XRP STEEM PRO BNB PPP BRAT BCD SALT KIN PLR RHOC PPT CVC CDT NEWB BAS PAY EOS DOM XRL LINK TNT BAT DNT SNT SKIN FUCK NIO HMQ IXT MSP IND DENT FUN BET'
port = 'CHC QRL BCC XRL RHOC KIN PPT SALT CDT BRAT PAY MBRS XRL EOS TAAS BNB HMQ OMG WINGS POE CTR LINK SKIN FUCK BAT CVC MCO SNT PPP CAT DOM BBT BET STX TIX DNT MGO VERI ICN NIO BQX TNT ZRX IXT MSP IND FUN LRC PLR'
port = 'CHC QRL BCC XRL RHOC BNB KIN PPT SALT CDT BRAT PAY MBRS XRL EOS TAAS BNB HMQ OMG WINGS POE CTR LINK SKIN FUCK BAT CVC MCO SNT PPP CAT DOM BBT BET STX TIX DNT MGO VERI ICN NIO BQX TNT ZRX IXT MSP IND FUN LRC PLR'
dfp = genDFP(cmc, port)
dfp

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 40, 5
import seaborn as sns
sns.set()

def clustermap(dfp, verbose=False):
    #sns.clustermap(dfp.corr(), center=0, cmap="vlag")#, row_colors=network_colors, col_colors=network_colors, linewidths=.75, figsize=(13, 13))
    #with p.option_context('display.max_rows', 400, 'display.max_columns', 4000, 'display.width', 1000000):
    #dfp.corr()#.loc[ ['bitcoin','ethereum'],:]
    dfpcl = dfp.columns.levels[1]
    #corrs = {}
    corrs = n.array([])
    print len(dfp.columns)
    for i in dfpcl:
        #with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
        #    #if verbose:
        #    #    print i
        cor = dfp.loc[:,(dfp.columns.levels[0],[i])].corr()        
        cor = cor.corr() #.loc[:,(dfp.columns.levels[0],[i])].corr()
        cor = cor.fillna(0)
        #corrs.update({i: cor})
        
        try:
            sns.clustermap(cor, center=0, cmap="vlag", figsize=(30,30))#, row_colors=network_colors, col_colors=network_colors, linewidths=.75, figsize=(13, 13))        
            plt.show()
            corrs = corrs + cor.get_values()
            ''
        except Exception as e: print e
        #with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
        #    if verbose:
        #        print corrs
        #print
    print corrs
    """
    #print corrs
    ck = corrs.keys()
    lis = n.array([0]*ck)
    
    for i in range(len(ck)):
        #print i
            lis[i] = corrs[ck[i]].get_values()
    with p.option_context('display.max_rows', 10, 'display.max_columns', 5, 'display.width', 1000000):
        print lis
    """
    #sns.clustermap(cor, center=0, cmap="vlag")#, row_colors=network_colors, col_colors=network_colors, linewidths=.75, figsize=(13, 13))        
    #plt.show()
    #return cor
clustermap(dfp, verbose=True)

In [ ]:
# Load the brain networks example dataset
df = sns.load_dataset("brain_networks", header=[0, 1, 2], index_col=0)

# Select a subset of the networks
used_networks = [1, 5, 6, 7, 8, 12, 13, 17]
used_columns = (df.columns.get_level_values("network")
                          .astype(int)
                          .isin(used_networks))
df = df.loc[:, used_columns]

# Create a categorical palette to identify the networks
network_pal = sns.husl_palette(8, s=.45)
#print network_pal
network_lut = dict(zip(map(str, used_networks), network_pal))

# Convert the palette to vectors that will be drawn on the side of the matrix
networks = df.columns.get_level_values("network")
network_colors = p.Series(networks, index=df.columns).map(network_lut)

# Draw the full plot
sns.clustermap(df.corr(), center=0, cmap="vlag",
               row_colors=network_colors, col_colors=network_colors,
               linewidths=.75, figsize=(13, 13))
#dfd = desc(df)
#df
df#.corr()

In [ ]:
dfm = cmc.glo('ethereum')
dfm

In [ ]:
li = list(df['id'].head(30))
print ' '.join(li)
for i in li:
    try:
        dfm = cmc.glo(i)
        plt.show()
    except: ''

In [ ]:
# show portfolio  charts
port = 'PPP XRL BBT SALT PPT TAAS SIFT PAY OMG KIN AIR PLR LINK RHOC CDT DNT RDN POWR CAT ENG PRO KNC SND AVT HMQ MGO ZRX VERI CVC EOS'
li = list(df.loc[port.split(' '),['id']]['id'])
#print ' '.join(li)
for i in list(li):
    try:
        cmc.glo(i)
        plt.show()
    except: ''

In [ ]:
# show portfolio eth1 charts
port = 'XRP STEEM PRO BNB PPP BRAT BCD SALT KIN PLR RHOC PPT CVC CDT NEWB BAS PAY EOS DOM XRL LINK TNT BAT DNT SNT SKIN FUCK NIO HMQ IXT MSP IND DENT FUN BET'
li = list(df.loc[port.split(' '),['id']]['id'])
#print ' '.join(li)
for i in list(li):
    try:
        cmc.glo(i)
        plt.show()
    except: ''

In [ ]:
# show portfolio eth2 charts
port = 'CHC QRL BCC XRL RHOC KIN PPT SALT CDT BRAT PAY MBRS XRL EOS TAAS BNB HMQ OMG WINGS POE CTR LINK SKIN FUCK BAT CVC MCO SNT PPP CAT DOM BBT BET STX TIX DNT MGO VERI ICN NIO BQX TNT ZRX IXT MSP IND FUN LRC PLR'
li = list(df.loc[port.split(' '),['id']]['id'])
#print ' '.join(li)
for i in list(li):
    try:
        cmc.glo(i)
        plt.show()
    except: ''

In [ ]:
nu = 10
for i in list(df.head(nu)['id']):
    cmc.glo(i)
    plt.show()
print '=-=-=-=-=-=-=-=-'
print '=-=-=-=-=-=-=-=-'
for i in list(df.tail(nu)['id']):
    cmc.glo(i)
    plt.show()

In [ ]:
#for i in 'bitcoin ethereum kin'.split(' '):
# b3 == pump n dump example
for i in 'b3coin'.split(' '):
    cmc.glo(i)